In [5]:
# with sliding window and mask
# HANDS + FACE ONLY (NO POSE)
# FIXED FEATURE DIMENSION

import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm
from collections import Counter

# ==============================
# CONFIG
# ==============================
DATA_DIR = r"E:\ASL_Citizen\NEW\Top_Classes_Landmarks_Preprocessed"
TARGET_FRAMES = 157
FEATURE_DIM = 438
BATCH_SIZE = 16
LR = 1e-3
EPOCHS = 60
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
PATIENCE = 7

print("Using device:", DEVICE)

# ==============================
# DATASET
# ==============================
class LandmarkDataset(Dataset):
    def __init__(self, files, labels):
        self.files = files
        self.labels = labels

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        x = np.load(self.files[idx]).astype(np.float32)
        x = torch.tensor(x).permute(1, 0)
        y = torch.tensor(self.labels[idx], dtype=torch.long)
        return x, y

# ==============================
# LOAD FILES & LABELS
# ==============================
files, labels = [], []

for f in os.listdir(DATA_DIR):
    if f.endswith(".npy") and "_mask" not in f:
        files.append(os.path.join(DATA_DIR, f))
        gloss = f.rsplit("_", 1)[0].split("_")[0]
        labels.append(gloss)

print(f"Found {len(files)} samples")

le = LabelEncoder()
y_encoded = le.fit_transform(labels)

# ==============================
# FILTER CLASSES WITH <2 SAMPLES
# ==============================
label_counts = Counter(y_encoded)
valid_idx = [i for i, y in enumerate(y_encoded) if label_counts[y] >= 2]

files = [files[i] for i in valid_idx]
y_encoded = y_encoded[valid_idx]

le = LabelEncoder()
y_encoded = le.fit_transform(y_encoded)
num_classes = len(le.classes_)

print(f"Classes after filtering: {num_classes}")

# ==============================
# SPLITS
# ==============================
files_train, files_tmp, y_train, y_tmp = train_test_split(
    files, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)

files_val, files_test, y_val, y_test = train_test_split(
    files_tmp, y_tmp, test_size=0.5, stratify=y_tmp, random_state=42
)

# ==============================
# CLASS WEIGHTS
# ==============================
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.arange(num_classes),
    y=y_train
)

class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(DEVICE)
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

# ==============================
# LOADERS
# ==============================
train_dataset = LandmarkDataset(files_train, y_train)
val_dataset   = LandmarkDataset(files_val, y_val)
test_dataset  = LandmarkDataset(files_test, y_test)

sample_weights = np.array([class_weights[y] for y in y_train])
sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=sampler)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# ==============================
# MODEL
# ==============================
class CNN1D(nn.Module):
    def __init__(self, input_features, num_classes):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(input_features, 256, 3, padding=1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Conv1d(256, 128, 3, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.05),

            nn.Conv1d(128, 64, 3, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.AdaptiveMaxPool1d(1)
        )
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.conv(x)
        x = x.squeeze(-1)
        return self.fc(x)

model = CNN1D(FEATURE_DIM, num_classes).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='max', factor=0.5, patience=3
)

# ==============================
# TRAINING (EXACT PRINT STYLE)
# ==============================
def train_model(model, train_loader, val_loader, epochs, patience):
    best_val_acc = 0
    counter = 0

    for epoch in range(epochs):
        model.train()
        running_loss = 0
        correct = 0
        total = 0

        for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]"):
            x, y = x.to(DEVICE), y.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * x.size(0)
            preds = outputs.argmax(1)
            correct += (preds == y).sum().item()
            total += y.size(0)

        train_loss = running_loss / total
        train_acc = correct / total

        # Validation
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for x_val, y_val in val_loader:
                x_val, y_val = x_val.to(DEVICE), y_val.to(DEVICE)
                preds = model(x_val).argmax(1)
                val_correct += (preds == y_val).sum().item()
                val_total += y_val.size(0)

        val_acc = val_correct / val_total
        scheduler.step(val_acc)

        print(
            f"Epoch {epoch+1}/{epochs} | "
            f"Train Loss: {train_loss:.4f} | "
            f"Train Acc: {train_acc:.4f} | "
            f"Val Acc: {val_acc:.4f}"
        )

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            counter = 0
            torch.save(model.state_dict(), "CNN_hands_face_sliding_mask_best_model.pth")
            print("  🔹 Saved best model")
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping triggered!")
                break

# ==============================
# EVALUATION
# ==============================
def evaluate_model(model, test_loader, label_encoder):
    model.load_state_dict(torch.load("CNN_hands_face_sliding_mask_best_model.pth"))
    model.eval()

    all_preds, all_labels = [], []

    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            preds = model(x).argmax(1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    acc = np.mean(np.array(all_preds) == np.array(all_labels))
    print(f"\nTest Accuracy: {acc:.4f}")

    preds_str = label_encoder.inverse_transform(all_preds)
    labels_str = label_encoder.inverse_transform(all_labels)

    print("\nClassification Report:")
    print(classification_report(labels_str, preds_str))

    print("\nConfusion Matrix:")
    print(confusion_matrix(labels_str, preds_str))

# ==============================
# RUN
# ==============================
train_model(model, train_loader, val_loader, EPOCHS, PATIENCE)
evaluate_model(model, test_loader, le)


Using device: cpu
Found 5845 samples
Classes after filtering: 146


Epoch 1/60 [Train]: 100%|████████████████████████████████████████████████████████████| 293/293 [00:18<00:00, 15.73it/s]


Epoch 1/60 | Train Loss: 4.7866 | Train Acc: 0.0282 | Val Acc: 0.0360
  🔹 Saved best model


Epoch 2/60 [Train]: 100%|████████████████████████████████████████████████████████████| 293/293 [00:18<00:00, 15.88it/s]


Epoch 2/60 | Train Loss: 3.9637 | Train Acc: 0.0740 | Val Acc: 0.0822
  🔹 Saved best model


...

Epoch 49/60 | Train Loss: 0.0465 | Train Acc: 0.9850 | Val Acc: 0.7705


Epoch 50/60 [Train]: 100%|███████████████████████████████████████████████████████████| 293/293 [00:21<00:00, 13.54it/s]


Epoch 50/60 | Train Loss: 0.0482 | Train Acc: 0.9829 | Val Acc: 0.7603


Epoch 51/60 [Train]: 100%|███████████████████████████████████████████████████████████| 293/293 [00:21<00:00, 13.84it/s]


Epoch 51/60 | Train Loss: 0.0420 | Train Acc: 0.9855 | Val Acc: 0.7688
Early stopping triggered!

Test Accuracy: 0.7846

Classification Report:
              precision    recall  f1-score   support

           0       0.40      0.29      0.33         7
           1       0.67      0.67      0.67         3
           2       0.75      1.00      0.86         3
           3       0.75      1.00      0.86         3
           4       0.75      0.75      0.75         4
           5       0.75      0.86      0.80         7
           6       1.00      0.67      0.80         3
           7       0.50      0.67      0.57         3
           8       1.00      1.00      1.00         3
           9       1.00      1.00      1.00         3
          10       1.00      0.67      0.80         3
          11       1.00      1.00      1.00         3
          12       1.00      0.67      0.80         3
          13       1.00      1.00      1.00         3
          14       0.75      1.00      0.86  